In [89]:
from pathlib import Path

import pandas as pd
import numpy as np

import argparse

from boostedhh import utils

In [ ]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [124]:
label_maps = {
    "hh": r"$\tau_h\tau_h$",
    "hm": r"$\tau_h\mu$",
    "he": r"$\tau_h e$",
    "bbtt": r"ggF HHbb$\tau\tau$",
    "vbfbbtt": r"VBF HHbb$\tau\tau$ SM",
    "vbfbbtt-k2v0": r"VBF HHbb$\tau\tau$ $\kappa_{2V}=0$",
}

all_years = ["2022", "2022EE", "2023", "2023BPix"]
all_signals = ["bbtt", "vbfbbtt", "vbfbbtt-k2v0"]

In [125]:
def csv_to_latex_efficiency(
    csv_paths: list[str],
    signal=str,
    channel=str,
    output_path: str = None,
    caption: str = None,
    label: str = None,
) -> str:
    """Convert a CSV file to LaTeX table format.

    Args:
        csv_path: Path to the CSV file
        output_path: Path to save the LaTeX output (optional)
        caption: Table caption (optional)
        label: Table label for referencing (optional)

    Returns:
        LaTeX table code as a string
    """

    dfs = [pd.read_csv(csv_path) for csv_path in csv_paths]

    latex_table = "\\begin{table}[htp]\n\\centering\n"

    if caption:
        latex_table += f"\\caption{{{caption}}}\n"

    # Add label if provided
    if label:
        latex_table += f"\\label{{{label}}}\n"

    i = 0
    for year, df in zip(all_years, dfs):
        # Generate table content
        latex_table += r"\resizebox*{1\textwidth}{!}{"
        latex_table += "\\begin{tabular}{" + "c" * len(df.columns) + "}\n"

        if i == 0:
            latex_table += "\\hline\n"
            latex_table += (
                r"\multicolumn{"
                + str(len(df.columns))
                + r"}{c}{\textbf{"
                + f"{label_maps[signal]}, {label_maps[channel]}"
                + r"}} \\"
            )
            latex_table += "\\hline\n"
            i += 1

        # Add headers
        latex_table += (
            r"\textbf{"
            + f"{year}"
            + r"} & "
            + " & ".join(str(val).replace("Trigger Efficiency", "") for val in df.columns[1:])
            + " \\\\\n"
        )
        latex_table += "\\hline\n"

        # Add data rows
        for _, row in df.iterrows():
            latex_table += (
                " & ".join(
                    [
                        str(row.iloc[0])
                        #   .replace(", XbbvsQCD > 0.95", " QCD cut")
                        #   .replace(", XbbvsQCDTop > 0.95", " QCDTop cut")
                        .replace(">", "$>$")
                        .replace("boosted", "")
                        .replace(" (", ", $p_T $ (")
                        .replace(")", ") GeV")
                        #   .replace("(> 250)","")
                        #   .replace("(>250, >200)","")
                    ]
                    + [(str(val) + "\%") for val in row.iloc[1:]]
                )
                + " \\\\\n"
            )

        latex_table += "\\hline\n"
        latex_table += "\\end{tabular}\n"
        latex_table += "}\n"

    latex_table += "\\end{table}\n\n"

    # Save to file if output path provided
    if output_path:
        with open(output_path, "w") as f:
            f.write(latex_table)

    return latex_table

In [144]:
def csv_to_latex_successive_removal(
    csv_paths: dict[str, dict[str, str]], channel: str, caption: str = None, label: str = None
) -> str:
    """Convert a CSV file to LaTeX table format.

    Args:
        csv_path: Path to the CSV file
        output_path: Path to save the LaTeX output (optional)
        caption: Table caption (optional)
        label: Table label for referencing (optional)

    Returns:
        LaTeX table code as a string
    """

    # [ f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/progressive_removal_{channel}_.csv" for year in all_years for signal in ["bbtt", "vbfbbtt", "vbfbbtt-k2v0"]]

    dfs = {
        signal: {year: pd.read_csv(csv_paths[signal][year]) for year in ["2022", "2023BPix"]}
        for signal in all_signals
    }

    latex_table = "\\begin{table}[H]\n\\centering\n"

    if caption:
        latex_table += f"\\caption{{{caption}}}\n"

    # Add label if provided
    if label:
        latex_table += f"\\label{{{label}}}\n"

    for signal in dfs:
        i = 0
        for year, df in dfs[signal].items():
            latex_table += r"\resizebox*{1\textwidth}{!}{"
            latex_table += "\\begin{tabular}{" + "c" * len(df.columns) + "}\n"
            latex_table += "\\hline\n"

            if i == 0:
                latex_table += "\\hline\n"
                latex_table += (
                    r"\multicolumn{"
                    + str(len(df.columns))
                    + r"}{c}{\textbf{"
                    + f"{label_maps[signal]}, {label_maps[channel]}"
                    + r"}} \\"
                )
                latex_table += "\\hline\n"
                i += 1

            # Add headers
            latex_table += (
                r"\textbf{"
                + f"{year}"
                + r"} & "
                + r" & ".join(
                    str(val)
                    .replace("_", "\_")
                    .replace("(", "")
                    .replace(")", "")
                    .replace("\_{", "_{")
                    .replace("Trigger Efficiency", "")
                    for val in df.columns[1:]
                )
                + " \\\\\n"
            )
            latex_table += "\\hline\n"

            # Add data rows
            for _, row in df.iterrows():
                latex_table += (
                    " & ".join(
                        str(val).replace("%", "\%")
                        #   .replace(", XbbvsQCD > 0.95", "QCD cut")
                        #   .replace(", XbbvsQCDTop > 0.95", "QCDTop cut")
                        #   .replace("(> 250)","")
                        #   .replace("(>250, >200)","")
                        .replace(">", "$>$")
                        .replace("boosted", "")
                        .replace(" (", ", $p_T $ (")
                        .replace(")", ") GeV")
                        for val in row
                    )
                    + " \\\\\n"
                )

            latex_table += "\\hline\n"
            latex_table += "\\end{tabular}\n"
            latex_table += "}\n"

    latex_table += "\\end{table}\n\n"

    return latex_table

In [145]:
# Successive subtraction: group by years and signals
channel = "hh"

input_paths = {
    signal: {
        year: f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/progressive_removal_{channel}_.csv"
        for year in ["2022", "2023BPix"]
    }
    for signal in all_signals
}

print(
    csv_to_latex_successive_removal(
        input_paths,
        channel,
        caption=f"Summary of signal efficiency for the signals {label_maps['bbtt']}, {label_maps['vbfbbtt']} and {label_maps['vbfbbtt-k2v0']} in two eras of years 2022 and 2023BPix for the channel {label_maps[channel]}. We only include two eras for brevity and because the other eras contain triggers only for a subset of runs. Going from left to right, each column represents the efficiency after cumulatively removing the indicated set of triggers. Each row lists a different preselection applied to the denominator.",
        label=f"table:trig_successive_removal_{channel}",
    )
)

\begin{table}[H]
\centering
\caption{Summary of signal efficiency for the signals ggF HHbb$\tau\tau$, VBF HHbb$\tau\tau$ SM and VBF HHbb$\tau\tau$ $\kappa_{2V}=0$ in two eras of years 2022 and 2023BPix for the channel $\tau_h\tau_h$. We only include two eras for brevity and because the other eras contain triggers only for a subset of runs. Going from left to right, each column represents the efficiency after cumulatively removing the indicated set of triggers. Each row lists a different preselection applied to the denominator.}
\label{table:trig_successive_removal_hh}
\resizebox*{1\textwidth}{!}{\begin{tabular}{cccccc}
\hline
\hline
\multicolumn{6}{c}{\textbf{ggF HHbb$\tau\tau$, $\tau_h\tau_h$}} \\\hline
\textbf{2022} & All & -PNetTauTau  & -QuadJet70\_50\_40\_35  & -QuadJet & -PFJet \\
\hline
1  jet, $p_T $ ($>$ 250) GeV & 83.8\% & 82.8\% & 78.7\% & 76.4\% & 76.1\% \\
2  jets, $p_T $ ($>$250, $>$200) GeV & 90.6\% & 89.0\% & 86.1\% & 84.5\% & 83.9\% \\
2  jets, $p_T $ ($>$250, $>$200) 

In [146]:
channel = "hm"

input_paths = {
    signal: {
        year: f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/progressive_removal_{channel}_.csv"
        for year in ["2022", "2023BPix"]
    }
    for signal in all_signals
}

print(
    csv_to_latex_successive_removal(
        input_paths,
        channel,
        caption=f"Summary of signal efficiency for the signals {label_maps['bbtt']}, {label_maps['vbfbbtt']} and {label_maps['vbfbbtt-k2v0']} in two eras of years 2022 and 2023BPix for the channel {label_maps[channel]}. We only include two eras for brevity and because the other eras contain triggers only for a subset of runs. Going from left to right, each column represents the efficiency after cumulatively removing the indicated set of triggers. Each row lists a different preselection applied to the denominator.",
        label=f"table:trig_successive_removal_{channel}",
    )
)

\begin{table}[H]
\centering
\caption{Summary of signal efficiency for the signals ggF HHbb$\tau\tau$, VBF HHbb$\tau\tau$ SM and VBF HHbb$\tau\tau$ $\kappa_{2V}=0$ in two eras of years 2022 and 2023BPix for the channel $\tau_h\mu$. We only include two eras for brevity and because the other eras contain triggers only for a subset of runs. Going from left to right, each column represents the efficiency after cumulatively removing the indicated set of triggers. Each row lists a different preselection applied to the denominator.}
\label{table:trig_successive_removal_hm}
\resizebox*{1\textwidth}{!}{\begin{tabular}{cccccc}
\hline
\hline
\multicolumn{6}{c}{\textbf{ggF HHbb$\tau\tau$, $\tau_h\mu$}} \\\hline
\textbf{2022} & All & -PNetTauTau  & -QuadJet70\_50\_40\_35  & -QuadJet & -PFJet \\
\hline
1  jet, $p_T $ ($>$ 250) GeV & 88.4\% & 87.8\% & 87.8\% & 87.8\% & 87.7\% \\
2  jets, $p_T $ ($>$250, $>$200) GeV & 91.0\% & 90.0\% & 90.0\% & 90.0\% & 89.7\% \\
2  jets, $p_T $ ($>$250, $>$200) GeV, X

In [147]:
channel = "he"

input_paths = {
    signal: {
        year: f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/progressive_removal_{channel}_.csv"
        for year in ["2022", "2023BPix"]
    }
    for signal in all_signals
}

print(
    csv_to_latex_successive_removal(
        input_paths,
        channel,
        caption=f"Summary of signal efficiency for the signals {label_maps['bbtt']}, {label_maps['vbfbbtt']} and {label_maps['vbfbbtt-k2v0']} in two eras of years 2022 and 2023BPix for the channel {label_maps[channel]}. We only include two eras for brevity and because the other eras contain triggers only for a subset of runs. Going from left to right, each column represents the efficiency after cumulatively removing the indicated set of triggers. Each row lists a different preselection applied to the denominator.",
        label=f"table:trig_successive_removal_{channel}",
    )
)

\begin{table}[H]
\centering
\caption{Summary of signal efficiency for the signals ggF HHbb$\tau\tau$, VBF HHbb$\tau\tau$ SM and VBF HHbb$\tau\tau$ $\kappa_{2V}=0$ in two eras of years 2022 and 2023BPix for the channel $\tau_h e$. We only include two eras for brevity and because the other eras contain triggers only for a subset of runs. Going from left to right, each column represents the efficiency after cumulatively removing the indicated set of triggers. Each row lists a different preselection applied to the denominator.}
\label{table:trig_successive_removal_he}
\resizebox*{1\textwidth}{!}{\begin{tabular}{cccccc}
\hline
\hline
\multicolumn{6}{c}{\textbf{ggF HHbb$\tau\tau$, $\tau_h e$}} \\\hline
\textbf{2022} & All & -PNetTauTau  & -QuadJet70\_50\_40\_35  & -QuadJet & -PFJet \\
\hline
1  jet, $p_T $ ($>$ 250) GeV & 86.7\% & 86.2\% & 86.2\% & 86.2\% & 86.1\% \\
2  jets, $p_T $ ($>$250, $>$200) GeV & 90.2\% & 89.5\% & 89.5\% & 89.5\% & 89.1\% \\
2  jets, $p_T $ ($>$250, $>$200) GeV, Xbb

In [108]:
# N-1 plots


def plot_latex_misc(input_dict=dict[str, list[str]]) -> str:
    """
    the lists are len 4 to pack in a 2x2 plot
    """

    signal = input_dict["signal"]
    year = input_dict["year"]
    channel = input_dict["channel"]
    fom = input_dict["fom"]
    isminus = input_dict["isminus"]

    # caption = ""
    paths = [
        f"figures/triggers/{year}/{signal}/{fom}_{channel}{'_'+isminus if isminus else ''}.pdf"
        for year, signal, channel, fom, isminus in zip(year, signal, channel, fom, isminus)
    ]

    latex_text = (
        r"""
    \begin{figure}[H]
        \centering
        \begin{tabular}{cc}
            \includegraphics[width=0.49\textwidth]{"""
        + paths[0]
        + r"""} &
            \includegraphics[width=0.49\textwidth]{"""
        + paths[1]
        + r"""} \\[1ex]
            \includegraphics[width=0.49\textwidth]{"""
        + paths[2]
        + r"""} &
            \includegraphics[width=0.49\textwidth]{"""
        + paths[3]
        + r"""}
            \label{fig}
        \end{tabular}
    \end{figure} 
    """
    )

    return latex_text


def plot_latex_yearsgrouped(signal=str, channel=str, fom=str, label=str, caption=str) -> str:

    # caption = ""
    paths = [
        f"figures/triggers/{year}/{signal}/{fom}_{channel}_minus.pdf"
        for year in ["2022", "2022EE", "2023", "2023BPix"]
    ]

    latex_text = (
        r"""
    \begin{figure}[H]
        \centering
        \begin{tabular}{cc}
            \includegraphics[width=0.49\textwidth]{"""
        + paths[0]
        + r"""} &
            \includegraphics[width=0.49\textwidth]{"""
        + paths[1]
        + r"""} \\[1ex]
            \includegraphics[width=0.49\textwidth]{"""
        + paths[2]
        + r"""} &
            \includegraphics[width=0.49\textwidth]{"""
        + paths[3]
        + r"""}
            \label{fig}
        \end{tabular}"""
    )

    if caption:
        latex_text += f"\n\\caption{{{caption}}}"
    if label:
        latex_text += f"\n\\label{{{label}}}"
    latex_text += "\n\\end{figure}"

    return latex_text

Start with N-1 plots and tables with hh

In [ ]:
# N-1 tables; mhh, yearsgrouped, hh, ggf
signal = "bbtt"
channel = "hh"
print(
    plot_latex_yearsgrouped(
        signal,
        channel,
        "mhh",
        label=f"fig:trig_yearsgrouped_{signal}_{channel}_mhh",
        caption=f"Trigger efficiencies for all 2022-2023 eras, after removing no or a single set of triggers for the {label_maps[signal]} signal for the channel {label_maps[channel]}, as a function of the dijet mass.",
    )
)
print(
    csv_to_latex_efficiency(
        [
            f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/trig_effs_{channel}.csv"
            for year in all_years
        ],
        signal,
        channel,
        caption=f"Summary of signal efficiency for the {label_maps[signal]} signal in the four eras of years 2022-2023 for the channel {label_maps[channel]} with all triggers and when removing a single set of triggers at a time. Each row lists a different preselection applied to the denominator, with the preselection efficiency given in the right column",
        label=f"table:trig_n1_{signal}_{channel}",
    )
)

In [ ]:
# N-1 tables; mhh, yearsgrouped, hh, vbf
signal = "vbfbbtt"
channel = "hh"
print(
    plot_latex_yearsgrouped(
        signal,
        channel,
        "mhh",
        label=f"fig:trig_yearsgrouped_{signal}_{channel}_mhh",
        caption=f"Trigger efficiencies for all 2022-2023 eras, after removing no or a single set of triggers for the {label_maps[signal]} signal for the channel {label_maps[channel]}, as a function of the dijet mass.",
    )
)
print(
    csv_to_latex_efficiency(
        [
            f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/trig_effs_{channel}.csv"
            for year in all_years
        ],
        signal,
        channel,
        caption=f"Summary of signal efficiency for the {label_maps[signal]} signal in the four eras of years 2022-2023 for the channel {label_maps[channel]} with all triggers and when removing a single set of triggers at a time. Each row lists a different preselection applied to the denominator, with the preselection efficiency given in the right column",
        label=f"table:trig_n1_{signal}_{channel}",
    )
)

In [ ]:
# N-1 tables; mhh, yearsgrouped, hh, vbfk2v0
signal = "vbfbbtt-k2v0"
channel = "hh"
print(
    plot_latex_yearsgrouped(
        signal,
        channel,
        "mhh",
        label=f"fig:trig_yearsgrouped_{signal}_{channel}_mhh",
        caption=f"Trigger efficiencies for all 2022-2023 eras, after removing no or a single set of triggers for the {label_maps[signal]} signal for the channel {label_maps[channel]}, as a function of the dijet mass.",
    )
)
print(
    csv_to_latex_efficiency(
        [
            f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/trig_effs_{channel}.csv"
            for year in all_years
        ],
        signal,
        channel,
        caption=f"Summary of signal efficiency for the {label_maps[signal]} signal in the four eras of years 2022-2023 for the channel {label_maps[channel]} with all triggers and when removing a single set of triggers at a time. Each row lists a different preselection applied to the denominator, with the preselection efficiency given in the right column",
        label=f"table:trig_n1_{signal}_{channel}",
    )
)

Now hm

In [ ]:
# N-1 tables; mhh, yearsgrouped, hm, ggf
signal = "bbtt"
channel = "hm"
print(
    plot_latex_yearsgrouped(
        signal,
        channel,
        "mhh",
        label=f"fig:trig_yearsgrouped_{signal}_{channel}_mhh",
        caption=f"Trigger efficiencies for all 2022-2023 eras, after removing no or a single set of triggers for the {label_maps[signal]} signal for the channel {label_maps[channel]}, as a function of the dijet mass.",
    )
)
print(
    csv_to_latex_efficiency(
        [
            f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/trig_effs_{channel}.csv"
            for year in all_years
        ],
        signal,
        channel,
        caption=f"Summary of signal efficiency for the {label_maps[signal]} signal in the four eras of years 2022-2023 for the channel {label_maps[channel]} with all triggers and when removing a single set of triggers at a time. Each row lists a different preselection applied to the denominator, with the preselection efficiency given in the right column",
        label=f"table:trig_n1_{signal}_{channel}",
    )
)

In [ ]:
# N-1 tables; mhh, yearsgrouped, hm, vbf
signal = "vbfbbtt"
channel = "hm"
print(
    plot_latex_yearsgrouped(
        signal,
        channel,
        "mhh",
        label=f"fig:trig_yearsgrouped_{signal}_{channel}_mhh",
        caption=f"Trigger efficiencies for all 2022-2023 eras, after removing no or a single set of triggers for the {label_maps[signal]} signal for the channel {label_maps[channel]}, as a function of the dijet mass.",
    )
)
print(
    csv_to_latex_efficiency(
        [
            f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/trig_effs_{channel}.csv"
            for year in all_years
        ],
        signal,
        channel,
        caption=f"Summary of signal efficiency for the {label_maps[signal]} signal in the four eras of years 2022-2023 for the channel {label_maps[channel]} with all triggers and when removing a single set of triggers at a time. Each row lists a different preselection applied to the denominator, with the preselection efficiency given in the right column",
        label=f"table:trig_n1_{signal}_{channel}",
    )
)

In [ ]:
# N-1 tables; mhh, yearsgrouped, hm, vbfk2v0
signal = "vbfbbtt-k2v0"
channel = "hm"
print(
    plot_latex_yearsgrouped(
        signal,
        channel,
        "mhh",
        label=f"fig:trig_yearsgrouped_{signal}_{channel}_mhh",
        caption=f"Trigger efficiencies for all 2022-2023 eras, after removing no or a single set of triggers for the {label_maps[signal]} signal for the channel {label_maps[channel]}, as a function of the dijet mass.",
    )
)
print(
    csv_to_latex_efficiency(
        [
            f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/trig_effs_{channel}.csv"
            for year in all_years
        ],
        signal,
        channel,
        caption=f"Summary of signal efficiency for the {label_maps[signal]} signal in the four eras of years 2022-2023 for the channel {label_maps[channel]} with all triggers and when removing a single set of triggers at a time. Each row lists a different preselection applied to the denominator, with the preselection efficiency given in the right column",
        label=f"table:trig_n1_{signal}_{channel}",
    )
)

Now he

In [ ]:
# N-1 tables; mhh, yearsgrouped, he, ggf
signal = "bbtt"
channel = "he"
print(
    plot_latex_yearsgrouped(
        signal,
        channel,
        "mhh",
        label=f"fig:trig_yearsgrouped_{signal}_{channel}_mhh",
        caption=f"Trigger efficiencies for all 2022-2023 eras, after removing no or a single set of triggers for the {label_maps[signal]} signal for the channel {label_maps[channel]}, as a function of the dijet mass.",
    )
)
print(
    csv_to_latex_efficiency(
        [
            f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/trig_effs_{channel}.csv"
            for year in all_years
        ],
        signal,
        channel,
        caption=f"Summary of signal efficiency for the {label_maps[signal]} signal in the four eras of years 2022-2023 for the channel {label_maps[channel]} with all triggers and when removing a single set of triggers at a time. Each row lists a different preselection applied to the denominator, with the preselection efficiency given in the right column",
        label=f"table:trig_n1_{signal}_{channel}",
    )
)

In [ ]:
# N-1 tables; mhh, yearsgrouped, he, vbf
signal = "vbfbbtt"
channel = "he"
print(
    plot_latex_yearsgrouped(
        signal,
        channel,
        "mhh",
        label=f"fig:trig_yearsgrouped_{signal}_{channel}_mhh",
        caption=f"Trigger efficiencies for all 2022-2023 eras, after removing no or a single set of triggers for the {label_maps[signal]} signal for the channel {label_maps[channel]}, as a function of the dijet mass.",
    )
)
print(
    csv_to_latex_efficiency(
        [
            f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/trig_effs_{channel}.csv"
            for year in all_years
        ],
        signal,
        channel,
        caption=f"Summary of signal efficiency for the {label_maps[signal]} signal in the four eras of years 2022-2023 for the channel {label_maps[channel]} with all triggers and when removing a single set of triggers at a time. Each row lists a different preselection applied to the denominator, with the preselection efficiency given in the right column",
        label=f"table:trig_n1_{signal}_{channel}",
    )
)

In [ ]:
# N-1 tables; mhh, yearsgrouped, he, vbfk2v0
signal = "vbfbbtt-k2v0"
channel = "he"
print(
    plot_latex_yearsgrouped(
        signal,
        channel,
        "mhh",
        label=f"fig:trig_yearsgrouped_{signal}_{channel}_mhh",
        caption=f"Trigger efficiencies for all 2022-2023 eras, after removing no or a single set of triggers for the {label_maps[signal]} signal for the channel {label_maps[channel]}, as a function of the dijet mass.",
    )
)
print(
    csv_to_latex_efficiency(
        [
            f"/home/users/lumori/bbtautau/plots/TriggerStudy/25Apr16/{year}/{signal}/trig_effs_{channel}.csv"
            for year in all_years
        ],
        signal,
        channel,
        caption=f"Summary of signal efficiency for the {label_maps[signal]} signal in the four eras of years 2022-2023 for the channel {label_maps[channel]} with all triggers and when removing a single set of triggers at a time. Each row lists a different preselection applied to the denominator, with the preselection efficiency given in the right column",
        label=f"table:trig_n1_{signal}_{channel}",
    )
)

In [ ]:
# httpt, yearsgrouped
for signal in ["bbtt", "vbfbbtt", "vbfbbtt-k2v0"]:
    for channel in ["hh", "hm", "he"]:
        print(
            plot_latex_yearsgrouped(
                signal,
                channel,
                "httpt",
                label=f"fig:trig_yearsgrouped_{signal}_{channel}_httpt",
                caption=f"Trigger efficiencies for all 2022-2023 eras, removing no or a single set of triggers for the {label_maps[signal]} signal for the channel {label_maps[channel]}, as a function of the $\tau\tau$ jet $p_T$.",
            )
        )

In [ ]:
# hbbpt, yearsgrouped
for signal in ["bbtt", "vbfbbtt", "vbfbbtt-k2v0"]:
    for channel in ["hh", "hm", "he"]:
        print(
            plot_latex_yearsgrouped(
                signal,
                channel,
                "hbbpt",
                label=f"fig:trig_yearsgrouped_{signal}_{channel}_hbbpt",
                caption=f"Trigger efficiencies after removing no or a single set of triggers for the {label_maps[signal]} signal for the channel {label_maps[channel]}, as a function of the $bb$ jet $p_T$.",
            )
        )

Here can plot some miscellaneous 4-plot-blocks as one pagers 

In [ ]:
p1 = {
    "signal": ["bbtt", "bbtt", "bbtt", "bbtt"],
    "year": ["2022", "2022EE", "2023", "2023BPix"],
    "channel": ["hh", "hm", "he", "hh"],
    "fom": ["hbbpt", "hbbpt", "hbbpt", "hbbpt"],
    "isminus": [0, 0, 0, 0],
}
print(plot_latex_misc(p1))

From here start with sensitivity study results


In [33]:
def masses_latex_channelgrouped() -> str:

    # caption = ""
    paths = [
        f"figures/Sensitivity_study/Sensitivity_25Apr25/25Apr25{ch}/mass_hhbbtt_tautau2022_2022EE_2023_2023BPix.pdf"
        for ch in ["hh", "hm", "he"]
    ]

    latex_text = (
        r"""
    \begin{figure}[H]
        \centering
            \includegraphics[width=0.8\textwidth]{"""
        + paths[0]
        + r"""} \
            \includegraphics[width=0.8\textwidth]{"""
        + paths[1]
        + r"""} \
            \includegraphics[width=0.8\textwidth]{"""
        + paths[2]
        + r"""}
        \label{fig:masses_hhbbtt_tautau_channelgrouped}
        \caption{Reconstructed mass distributions of \textit{bb} and $\tau\tau$ AK8 jets for the three \textit{ggF} channels. The reconstructions are performed with SoftDrop, ParticleNet, and the Global Particle Transformer (GloParT) Resonant and Visible algorithms.}
    \end{figure} 
    """
    )

    return latex_text

In [ ]:
print(masses_latex_channelgrouped())

In [55]:
def optimization_latex_onepager() -> str:

    # caption = ""
    paths = [
        f"figures/Sensitivity_study/Sensitivity_25Apr25/25Apr25{ch}/sig_bkg_opt_2022_2022EE_2023_2023BPix_B={b}_abcd.pdf"
        for ch in ["hh", "hm", "he"]
        for b in [1, 2]
    ]

    latex_text = (
        r"""
    \begin{figure}[H]
        \begin{tabular}{cc}
            \includegraphics[width=0.45\textwidth]{"""
        + paths[0]
        + r"""} &
            \includegraphics[width=0.45\textwidth]{"""
        + paths[1]
        + r"""} \\[1ex]
            \includegraphics[width=0.45\textwidth]{"""
        + paths[2]
        + r"""} &
            \includegraphics[width=0.45\textwidth]{"""
        + paths[3]
        + r"""} \\[1ex]
            \includegraphics[width=0.45\textwidth]{"""
        + paths[4]
        + r"""} &
            \includegraphics[width=0.45\textwidth]{"""
        + paths[5]
        + r"""}
        \end{tabular}
            \label{fig:masses_hhbbtt_tautau_channelgrouped}
        \caption{Optimization of signal region condition on GloParT $bb$ and $\tautau$ discriminant scores defined against QCD and QCD + $t\bar{t}$. Color axis indicates the signal efficiency of a selection criterion corresponding to discriminant values on the axes. The red dot indicates the maximum signal yield constraining the total background yield as indicated, while the blue dot corresponds to the selection that minimizes $2 \times \sqrt{b} / s$ The first row describes the """
        + label_maps["hh"]
        + """, the second row describes the """
        + label_maps["hm"]
        + """, and the third row describes the """
        + label_maps["he"]
        + """ signal. }
    \end{figure} 
    """
    )

    return latex_text

In [ ]:
print(optimization_latex_onepager())